In [1]:
# Local clone
! git clone https://github.com/nanopiero/CML_processing_by_ML.git

fatal: destination path 'CML_processing_by_ML' already exists and is not an empty directory.


In [2]:
# Imports
from os.path import join, isdir, isfile
from os import listdir as ls
import copy
import torch
import numpy as np

import sys
sys.path.append('CML_processing_by_ML')
from src.utils.simulation import create_dataloader
import src.utils.architectures_fcn
from src.utils.architectures import load_archi
from src.utils.architectures_fcn import UNet_causal_5mn_atrous, UNet_causal_5mn_atrous_rescale

In [3]:
# Dictionary with pseudo "distances" (distances between two antennnas) for 1,000 pseudo CML ids.
idx2distance = {i: 0.2 +  1.8 * torch.rand((1,)).item() for i in range(0, 1000)}
duration = 4096  # Duration for each tensor pair
batch_size = 100  # Number of samples per batch
dataloader = create_dataloader(duration, idx2distance, batch_size)

In [4]:
# Here we samples 64 ground-truth rainy processes and their noisy counterpart
# A rainy process is modeled by a 1-d Neymann-Scott process
# The Intensity of the Poisson process for parent events is 0.05 x distance
# The resulting rainy process is divided by the distance to give the "ground truth"
# while it is corrupted through the following steps to yield "noisy_series":
# - applying a non linear conversion to an attenuation in db
# - applying a "wet antenna convolution filter" (kind of sliding mean)
# - adding a high (gaussian noise with non linear dependance of sigma wrt the intensity)
# - adding a low frequency random processes

for batch_idx, (idxs, dists, ground_truths, noisy_series) in enumerate(dataloader):
  if batch_idx == 0:
    break

In [ ]:
import matplotlib.pyplot as plt
sigma = 2
for k in range(5):
  print(idxs[k], dists[k])
  plt.figure(figsize=(10, 6))
  plt.plot(np.arange(duration), ground_truths[k], label='ground_truth')
  plt.plot(np.arange(duration), noisy_series[k], label='predictor')
  plt.title(f'Inputs and Targets for the CML n°{idxs[k].item():.0f} (ditance: {dists[k].item():.2f})')
  plt.xlabel('Time (minutes)')
  plt.ylabel('Event Density')
  plt.ylim(-1,6)
  plt.legend()
  plt.show()

In [ ]:
# Same weights for all CMLs

arch = "UNet_causal_5mn_atrous"
nchannels = 1
nclasses = 1 # Regression only
dilation = 2
atrous_rates=[6, 12, 18] #, 24, 30, 36, 42]
additional_parameters = 0

model = load_archi(arch, nchannels, nclasses, size=64, dilation=1,
                   atrous_rates=atrous_rates, fixed_cumul=False,
                   additional_parameters=additional_parameters)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# model = UNet(1, 1, 16).to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.MSELoss()

num_epochs = 300  # Adjust based on your needs

model.train()
for epoch in range(num_epochs):
    running_loss = 0.0
    for batch_idx, (idxs, dists, ground_truths, noisy_series) in enumerate(dataloader):
        inputs, targets = noisy_series.to(device), \
                          ground_truths.to(device)

        # Add the channel's dim
        inputs = inputs.unsqueeze(1)
        targets = targets.unsqueeze(1)

        # Zeroing gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # Backward and optimize
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(dataloader):.4f}')

In [ ]:
import matplotlib.pyplot as plt

def visualize_predictions(model, data_loader, num_samples=1):
    model.eval()
    L = 1000
    with torch.no_grad():
        for batch_idx, (idxs, dists, ground_truths, noisy_series) in enumerate(dataloader):

            inputs = noisy_series.to(device).unsqueeze(1).float()  # Adjust input dimensions
            outputs = model(inputs).cpu()

            for i in range(num_samples):
                plt.figure(figsize=(16, 4))
                # plt.plot(noisy_series[i].squeeze(), label='input')
                plt.plot(ground_truths[i,:L].squeeze(), label='Observation')
                plt.plot(outputs[i].squeeze()[:L], label='Prediction', linestyle='--')
                plt.legend()
                plt.show()
            break  # Just show the first batch

visualize_predictions(model, dataloader, num_samples=5)


In [ ]:
# How to do better ?

In [8]:
# sol 1 : UNet_causal_5mn_atrous_rescale: adds a scaling parameter for each CML

arch = "UNet_causal_5mn_atrous_rescale"
nchannels = 1
nclasses = 1 # Regression only
dilation = 2
atrous_rates=[6, 12, 18] #, 24, 30, 36, 42]
additional_parameters = 1005

model = load_archi(arch, nchannels, nclasses, size=64, dilation=1,
                   atrous_rates=atrous_rates, fixed_cumul=False,
                   additional_parameters=additional_parameters)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# model = UNet(1, 1, 16).to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.MSELoss()

num_epochs = 50  # Adjust based on your needs

model.train()
for epoch in range(num_epochs):
    running_loss = 0.0
    for batch_idx, (idxs, dists, ground_truths, noisy_series) in enumerate(dataloader):
        inputs, targets = noisy_series.to(device), \
                          ground_truths.to(device)

        # Add the channel's dim
        inputs = inputs.unsqueeze(1)
        targets = targets.unsqueeze(1)

        # Zeroing gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)

        outputs, p = model(inputs, indices=idxs.to(device))
        outputs[:,:,:] *= p[5:].view(outputs.shape[0],1,1)

        loss = criterion(outputs, targets)

        # Backward and optimize
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(dataloader):.4f}')

Epoch [1/50], Loss: 0.6667
Epoch [2/50], Loss: 0.2918
Epoch [3/50], Loss: 0.2657
Epoch [4/50], Loss: 0.2516
Epoch [5/50], Loss: 0.2470
Epoch [6/50], Loss: 0.2376
Epoch [7/50], Loss: 0.2327
Epoch [8/50], Loss: 0.2272
Epoch [9/50], Loss: 0.2220
Epoch [10/50], Loss: 0.2041
Epoch [11/50], Loss: 0.1878
Epoch [12/50], Loss: 0.1781
Epoch [13/50], Loss: 0.1728
Epoch [14/50], Loss: 0.1630
Epoch [15/50], Loss: 0.1610
Epoch [16/50], Loss: 0.1571
Epoch [17/50], Loss: 0.1582
Epoch [18/50], Loss: 0.1555
Epoch [19/50], Loss: 0.1561
Epoch [20/50], Loss: 0.1538
Epoch [21/50], Loss: 0.1518
Epoch [22/50], Loss: 0.1504
Epoch [23/50], Loss: 0.1490
Epoch [24/50], Loss: 0.1499
Epoch [25/50], Loss: 0.1509
Epoch [26/50], Loss: 0.1468
Epoch [27/50], Loss: 0.1507
Epoch [28/50], Loss: 0.1467
Epoch [29/50], Loss: 0.1451
Epoch [30/50], Loss: 0.1464
Epoch [31/50], Loss: 0.1445
Epoch [32/50], Loss: 0.1431
Epoch [33/50], Loss: 0.1430
Epoch [34/50], Loss: 0.1433
Epoch [35/50], Loss: 0.1462
Epoch [36/50], Loss: 0.1425
E

In [87]:
# sol 2 (step 1): with UNet_causal_5mn_multiplicative_rescale,
# the scaling parameter
# is yielded by specific perceptrons (one per CML)

arch = "UNet_causal_5mn_atrous_multiplicative_rescale"
nchannels = 1
nclasses = 5
dilation = 2
atrous_rates=[6, 12, 18] #, 24, 30, 36, 42]
additional_parameters = 0

model = load_archi(arch, nchannels, nclasses, size=64, dilation=1,
                   atrous_rates=atrous_rates, fixed_cumul=False,
                   additional_parameters=additional_parameters)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


# model = UNet(1, 1, 16).to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))
# model.freeze_specific_parts()

criterion = torch.nn.MSELoss()

num_epochs = 50  # Adjust based on your needs

model.train()

for epoch in range(num_epochs):
    running_loss = 0.0
    for batch_idx, (idxs, dists, ground_truths, noisy_series) in enumerate(dataloader):
        inputs, targets = noisy_series.to(device), \
                          ground_truths.to(device)

        use_first_network = torch.rand(idxs.shape, device=inputs.device) > 0.75
        idxs[use_first_network] = -1
        # Add the channel's dim
        inputs = inputs.unsqueeze(1)
        targets = targets.unsqueeze(1)

        # Zeroing gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs, idxs.to(device))

        loss = criterion(outputs, targets)

        # Backward and optimize
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(dataloader):.4f}')

Epoch [1/50], Loss: 0.4377
Epoch [2/50], Loss: 0.2800
Epoch [3/50], Loss: 0.2586
Epoch [4/50], Loss: 0.2482
Epoch [5/50], Loss: 0.2349
Epoch [6/50], Loss: 0.2247
Epoch [7/50], Loss: 0.2139
Epoch [8/50], Loss: 0.2196
Epoch [9/50], Loss: 0.2121
Epoch [10/50], Loss: 0.1996
Epoch [11/50], Loss: 0.1839
Epoch [12/50], Loss: 0.1772
Epoch [13/50], Loss: 0.1666
Epoch [14/50], Loss: 0.1610
Epoch [15/50], Loss: 0.1561
Epoch [16/50], Loss: 0.1551
Epoch [17/50], Loss: 0.1540
Epoch [18/50], Loss: 0.1502
Epoch [19/50], Loss: 0.1574
Epoch [20/50], Loss: 0.1509
Epoch [21/50], Loss: 0.1464
Epoch [22/50], Loss: 0.1464
Epoch [23/50], Loss: 0.1450
Epoch [24/50], Loss: 0.1418
Epoch [25/50], Loss: 0.1457
Epoch [26/50], Loss: 0.1422
Epoch [27/50], Loss: 0.1380
Epoch [28/50], Loss: 0.1385
Epoch [29/50], Loss: 0.1388
Epoch [30/50], Loss: 0.1401
Epoch [31/50], Loss: 0.1381
Epoch [32/50], Loss: 0.1376
Epoch [33/50], Loss: 0.1371
Epoch [34/50], Loss: 0.1384
Epoch [35/50], Loss: 0.1398
Epoch [36/50], Loss: 0.1357
E

In [88]:
# sol 2 (step 2): the generic part is freezed while the specific perceptrons
# are fine tuned
num_epochs = 20

model.unfreeze_specific_parts()
model.freeze_generic_parts()

for epoch in range(num_epochs):
    running_loss = 0.0
    for batch_idx, (idxs, dists, ground_truths, noisy_series) in enumerate(dataloader):
        inputs, targets = noisy_series.to(device), \
                          ground_truths.to(device)

        # Add the channel's dim
        inputs = inputs.unsqueeze(1)
        targets = targets.unsqueeze(1)

        # Zeroing gradients
        optimizer.zero_grad()
        # Forward pass
        outputs = model(inputs, idxs.to(device))

        loss = criterion(outputs, targets)

        # Backward and optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(dataloader):.4f}')

Epoch [1/20], Loss: 0.1262
Epoch [2/20], Loss: 0.1258
Epoch [3/20], Loss: 0.1263
Epoch [4/20], Loss: 0.1249
Epoch [5/20], Loss: 0.1247
Epoch [6/20], Loss: 0.1255
Epoch [7/20], Loss: 0.1252
Epoch [8/20], Loss: 0.1244
Epoch [9/20], Loss: 0.1249
Epoch [10/20], Loss: 0.1240
Epoch [11/20], Loss: 0.1232
Epoch [12/20], Loss: 0.1235
Epoch [13/20], Loss: 0.1232
Epoch [14/20], Loss: 0.1245
Epoch [15/20], Loss: 0.1239
Epoch [16/20], Loss: 0.1232
Epoch [17/20], Loss: 0.1234
Epoch [18/20], Loss: 0.1233
Epoch [19/20], Loss: 0.1242
Epoch [20/20], Loss: 0.1238


In [ ]:
# To pull and reload, if needed:
! cd CML_processing_by_ML ; git pull ; cd ..

import importlib
importlib.reload(src.utils.architectures_fcn)
importlib.reload(src.utils.architectures)
from src.utils.architectures_fcn import  UNet_causal_5mn_atrous_multiplicative_rescale
from src.utils.architectures import  load_archi